<center><h1>TLDA</h1></center>

<img src="tlda.png">

1. `theta_u` : pattern distribution of user *u*.
2. `phi_t` : pattern distribution of time *t*.
3. `psi_z` : venue category distribution of pattern *z*

Analogous to the traditional LDA : `document` - `user/time`, `topic` - `cultural pattern`, and `word` - `venue`.

<h2>Data Extraction</h2>

In [39]:
from collections import defaultdict
import numpy as np

In [40]:
# Month(str): Jan Feb [Mar not included] Apr May Jun Jul Aug Sep Oct Nov Dec
# day: Mon, Tue, Wed, Thu, Fri, Sat, Sun
# Hour(int): 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23


# data format: (user1 (venue_category_1, time_1), (venue_category_2, time_2), (venue_category_3, time_3))
# time format: month-day-hour; Example: (Oct-Wed-13)
checkin_data = defaultdict(list)

nyc = open("nyc.txt")

for checkin in nyc:
    checkin_time = (checkin.split('\t')[-1]).split(' ')
    
    # time in the correct format
    time = checkin_time[1] + '-' + checkin_time[0] + '-' + checkin_time[3].split(':')[0]
    
    # corresponding venue category
    venue_category = checkin.split('\t')[3]
    
    # one checkin tuple of this user
    single_checkin = (venue_category, time)

    #user id
    user = checkin.split('\t')[0]
    
    checkin_data[user].append(single_checkin)

        

days = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', 
        '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']


In [43]:
print("Number of users: {}".format(len(checkin_data)))

Number of users: 1083


<h2> Basic info </h2>
There are a total of 1083 users, with id from 1 to 1083.